# Ultimate Guitar Scrape - 4
## Scrape any missed tabs and collate
*Take all tabs missed from individual scrape and rescrape, then collate all CSVs together*

----

**Final Project for Data & Databases**

**C.J. Robinson**

**Fall 2024**

-----
### Scrape song pages

In [3]:
import re
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from playwright.async_api import async_playwright
import numpy as np

In [6]:
# read in all bad urls
bad_urls = pd.DataFrame()

bad_urls = pd.concat([pd.read_csv("bad_urls_1k_df.csv"),
                     pd.read_csv("bad_urls_2k_df.csv"),
                     pd.read_csv("bad_urls_3k_df.csv"),
                     pd.read_csv("bad_urls_4k_df.csv")], ignore_index=True)

,bad_urls
0,https://tabs.ultimate-guitar.com/tab/guns-n-ro...
1,https://tabs.ultimate-guitar.com/tab/billie-ei...
2,#
3,#
4,#


### Loop through all URLs missed

In [9]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()

retry_song_list = []

for url in bad_urls['bad_urls']:
    my_url = url
    
    try:
        await page.goto(my_url, timeout=120000)
        time.sleep(3)
        #scroll down to get chords to actually load
        await page.evaluate("window.scrollTo(0, 2000)") 
        time.sleep(3)
        html = await page.content()
        
        soup_doc = BeautifulSoup(html, "html.parser")
        
        song_dict = {}
        song_dict['link'] = my_url
        
        #there's a bunch of metadata that may or may not be in the header
        #since it depends on the author
        #go through anything in that header and pull out metadata name and value
        try:
            for tag in soup_doc.find_all("th", class_ = "ZvOWv"):
                data_type = tag.text.lower()
                song_dict[data_type] = tag.next_sibling()[0].text
        except:
            print("No meta data")
            print(my_url)
        
        # only grab chord elements if it is chords, not tabs
        try:
            chord_list = []
            #pull all chords
            for chords in soup_doc.find_all("span", class_="_Oy28"):
                chord_list.append(chords.text)
        
            #get unique chord list
            song_dict['chord_list'] = list(set(chord_list))
            #also grab full text...just in case
            song_dict['full_text'] = soup_doc.find("pre", class_="tK8GG Ty_RP").text
        except:
            print("No chords")
        
        # get raw text of contributions, will regex later
        try:
            song_dict['contributors'] = soup_doc.find("span", class_="zku_4").text
        except:
            print("No contributors" + my_url)
        
        # get popularity raw, also will regex later
        try:
            song_dict['popularity'] = soup_doc.find("div", class_="_apVM").text
        except:
            print("No popularity" + my_url)
    
        retry_song_list.append(song_dict)

    except:
        print("error with " + url)
    
retry_song_list = pd.json_normalize(retry_song_list)
retry_song_list.to_csv("retry_song_list.csv", index = False)

Future exception was never retrieved
future: <Future finished exception=Exception('Connection closed while reading from the driver')>
Exception: Connection closed while reading from the driver
Future exception was never retrieved
future: <Future finished exception=Exception('Connection closed while reading from the driver')>
Exception: Connection closed while reading from the driver


error with #
error with #
error with #
error with #
error with #
error with #
error with #
error with #
error with #
error with #
error with #
No chords
No chords
No chords
No chords
No contributorshttps://tabs.ultimate-guitar.com/tab/thin-lizzy/got-to-give-it-up-guitar-pro-545477
No chords
No chords
No chords
No chords
No chords
No chords
No chords
No chords
error with #
No chords
No contributorshttps://tabs.ultimate-guitar.com/tab/audioslave/like-a-stone-tabs-45721
No chords
No chords
No chords
No chords
No chords


### Read in all CSVs

In [13]:
top_songs = pd.read_csv("top_songs.csv")
#drop the index column
top_songs = top_songs.drop(top_songs.columns[0], axis=1) 

song_metadata = pd.concat([pd.read_csv("retry_song_list.csv"),
                       pd.read_csv("top_songs_metadata_1k.csv"),
                       pd.read_csv("top_songs_metadata_2k.csv"),
                       pd.read_csv("top_songs_metadata_3k.csv"),
                       pd.read_csv("top_songs_metadata_4k.csv")])

artist_data = pd.read_csv('artist_data.csv')
artist_data = artist_data.rename(columns={'type':'person_group'})

### Join on original

In [14]:
df = top_songs.merge(song_metadata, how = "left", left_on = "song_link", right_on = "link") 
df = df.merge(artist_data, how = "left", left_on = "artist", right_on = "artist")

# clean up some columns
df['contributors'] = df['contributors'].str.extract(r'(\d+)')
df['favorites'] = df['popularity'].str.extract(r'\d[\d,]* views, added to favorites (\d[\d,]*) times')
df['favorites'] = df['favorites'].str.replace(',', '')

# change to int
df['ratings'] = df['ratings'].str.replace(',', '')
df['hits'] = df['hits'].str.replace(',', '')

# get song title wihout version
df['song_no_vers'] = df['song'].str.replace(r'\s*\(ver\s*\d+\)\s*', '', regex=True)

#get version (if no version, it's the first version)
df['version'] = df['song'].str.extract(r'ver\s*(\d+)', expand=False).fillna(1)

#change capo to int
df['capo'] = df['capo: '].str.extract(r'(^\d+)')

# if there are 0 stars, it's just that there's no ratings
df['star_count'] = df['star_count'].replace(0, np.nan)

df.rename(columns= {
    'key: ':'key', 
    'tuning: ':'tuning',
    'difficulty: ':'difficulty'}, inplace=True)

# get rid of some unnecessary columns
df = df.drop(columns=['file format: ', 'filesize: ', 'instuments: ', 'capo: '], axis=1) 


In [15]:
df.head(10)

,rank,artist,artist_list,song,ratings,hits,type,song_link,artist_link,star_count,...,id,person_group,gender,country,start_date,tags,favorites,song_no_vers,version,capo
0,1,Ed Sheeran,['Ed Sheeran'],Perfect,48238,41205313,chords,https://tabs.ultimate-guitar.com/tab/ed-sheera...,https://www.ultimate-guitar.com/artist/ed_shee...,5.0,...,b8a7c51f-362c-4dcb-a259-bc6e0095f0a6,Person,male,GB,1991-02-17,"['pop', 'guitarist', 'folk', 'singer-songwrite...",620778,Perfect,1,1
1,2,Jeff Buckley,['Jeff Buckley'],Hallelujah (ver 2),54484,39807305,chords,https://tabs.ultimate-guitar.com/tab/jeff-buck...,https://www.ultimate-guitar.com/artist/jeff_bu...,5.0,...,e6e879c0-3d56-4f12-b3c5-3ce459661a8e,Person,male,US,1966-11-17,"['rock', 'alternative rock', 'american', 'blue...",502048,Hallelujah,2,1
2,3,Elvis Presley,['Elvis Presley'],Cant Help Falling In Love,32809,33890059,chords,https://tabs.ultimate-guitar.com/tab/elvis-pre...,https://www.ultimate-guitar.com/artist/elvis_p...,5.0,...,01809552-4f87-45b0-afff-2c6f0730a3be,Person,male,US,1935-01-08,"['rock', 'pop', 'american', 'blues', 'singer',...",665632,Cant Help Falling In Love,1,2
3,4,Passenger,['Passenger'],Let Her Go,24248,31904817,chords,https://tabs.ultimate-guitar.com/tab/passenger...,https://www.ultimate-guitar.com/artist/passeng...,5.0,...,186e216a-2f8a-41a1-935f-8e30c018a8fe,Person,male,GB,1984-05-17,"['alternative indy folk', 'alternative rock', ...",631253,Let Her Go,1,7
4,5,John Legend,['John Legend'],All Of Me,26699,29790560,chords,https://tabs.ultimate-guitar.com/tab/john-lege...,https://www.ultimate-guitar.com/artist/john_le...,5.0,...,75a72702-a5ef-4513-bca5-c5b944903546,Person,male,US,1978-12-28,"['american', 'usa', 'soul', 'américain', 'neo ...",474872,All Of Me,1,1
5,6,Jason Mraz,['Jason Mraz'],Im Yours (ver 11),16070,24611968,chords,https://tabs.ultimate-guitar.com/tab/jason-mra...,https://www.ultimate-guitar.com/artist/jason_m...,5.0,...,82eb8936-7bf6-4577-8320-a2639465206d,Person,male,US,1977-06-23,"['adult contemporary', 'rock', 'pop', 'alterna...",487492,Im Yours,11,4
6,7,Led Zeppelin,['Led Zeppelin'],Stairway To Heaven,14093,24568608,tab,https://tabs.ultimate-guitar.com/tab/led-zeppe...,https://www.ultimate-guitar.com/artist/led_zep...,5.0,...,678d88b2-87b0-403b-b63d-5da7465aecc3,Group,NaN,GB,1968,"['rock', 'heavy metal', 'folk', 'metal', 'amer...",257646,Stairway To Heaven,1,NaN
7,8,Radiohead,['Radiohead'],Creep,32994,22829081,chords,https://tabs.ultimate-guitar.com/tab/radiohead...,https://www.ultimate-guitar.com/artist/radiohe...,5.0,...,a74b1b7f-71a5-4011-9441-d0b5e4122711,Group,NaN,GB,1991,"['rock', 'electronic', 'post-rock', 'alternati...",400246,Creep,1,NaN
8,9,Ed Sheeran,['Ed Sheeran'],Thinking Out Loud,17356,22366807,chords,https://tabs.ultimate-guitar.com/tab/ed-sheera...,https://www.ultimate-guitar.com/artist/ed_shee...,5.0,...,b8a7c51f-362c-4dcb-a259-bc6e0095f0a6,Person,male,GB,1991-02-17,"['pop', 'guitarist', 'folk', 'singer-songwrite...",459590,Thinking Out Loud,1,2
9,10,Vance Joy,['Vance Joy'],Riptide (ver 2),28313,21110362,chords,https://tabs.ultimate-guitar.com/tab/vance-joy...,https://www.ultimate-guitar.com/artist/vance_j...,5.0,...,df36f6db-5987-46ed-9d02-0cf36ed4e060,Person,male,AU,1987-12-01,"['singer-songwriter', 'alternative rock', 'fol...",558762,Riptide,2,1


In [16]:
df.to_csv("top_songs_full.csv", index=False)